<a href="https://colab.research.google.com/github/rukmals/crowd-monitoring-system-model-development/blob/main/Drive_Mount_and_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls "/content/drive/MyDrive/GCC/Part 0"

scene_00_0  scene_01_3	scene_03_2  scene_05_1	scene_07_0  scene_08_3
scene_00_1  scene_02_0	scene_03_3  scene_05_2	scene_07_1  scene_09_0
scene_00_2  scene_02_1	scene_04_0  scene_05_3	scene_07_2  scene_09_1
scene_00_3  scene_02_2	scene_04_1  scene_06_0	scene_07_3  scene_09_2
scene_01_0  scene_02_3	scene_04_2  scene_06_1	scene_08_0  scene_09_3
scene_01_1  scene_03_0	scene_04_3  scene_06_2	scene_08_1
scene_01_2  scene_03_1	scene_05_0  scene_06_3	scene_08_2


In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [3]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  | Proc size: 118.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [4]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import glob
from sklearn.model_selection import train_test_split
import json

import sys
import warnings
# import from library
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import argparse
import json
import cv2
import time
from torchvision import models

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, \
                stride=1, NL='relu', same_padding=False, bn=False, dilation=1):
        super(Conv2d, self).__init__()
        padding = int((kernel_size - 1) // 2) if same_padding else 0
        self.conv = []
        if dilation==1:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, dilation=dilation)
        else:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=dilation, dilation=dilation)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else nn.Identity()
        if NL == 'relu' :
            self.relu = nn.ReLU(inplace=True)
        elif NL == 'prelu':
            self.relu = nn.PReLU()
        else:
            self.relu = None

    def forward(self, x):
      x = self.conv(x)
      if self.bn is not None:
          x = self.bn(x)
      if self.relu is not None:
          x = self.relu(x)   
      return x
  
# the module definition for the multi-branch in the density head
class MultiBranchModule(nn.Module):
    def __init__(self, in_channels, sync=False):
        super(MultiBranchModule, self).__init__()
        self.branch_column1_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column1_2 = BasicConv2d(in_channels//2, in_channels, kernel_size=1, sync=sync)

        self.branch_column2_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column2_2 = BasicConv2d(in_channels // 2, in_channels, kernel_size=(3, 3), padding=(1, 1), sync=sync)

        self.branch_column3_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column3_2 = BasicConv2d(in_channels // 2, in_channels, kernel_size=5, padding=2, sync=sync)

    def forward(self, x):
        branch_1 = self.branch_column1_1(x)
        branch_1 = self.branch_column1_2(branch_1)

        branch_2 = self.branch_column2_1(x)
        branch_2 = self.branch_column2_2(branch_2)

        branch_3 = self.branch_column3_1(x)
        branch_3 = self.branch_column3_2(branch_3)

        outputs = [branch_1, branch_2, branch_3, x]
        return torch.cat(outputs, 1)

# the module definition for the basic conv module
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, sync=False, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        if sync:
            # for sync bn
            print('use sync inception')
            self.bn = nn.SyncBatchNorm(out_channels, eps=0.001)
        else:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [48]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image

def load_data(img_path,train = True):
    gt_path = img_path.replace('.png','.h5').replace('pngs','GT')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path, 'r')
    target = np.asarray(gt_file['density'])
    target = cv2.resize(target,(int(target.shape[1]/2.25), int(target.shape[0]/2.25)),interpolation = cv2.INTER_CUBIC)*64
    return img,target
"""
create a list of file (full directory)
"""

def create_training_image_list(data_path):
    """
    create a list of absolutely path of jpg file
    :param data_path: must contain subfolder "images" with *.jpg  (example ShanghaiTech/part_A/train_data/)
    :return:
    """
    DATA_PATH = data_path
    image_path_list = glob.glob(os.path.join(DATA_PATH, "pngs", "*.png"))
    return image_path_list


def get_train_val_list(data_path):
    DATA_PATH = data_path
    scences = ['scene_00_0','scene_00_1','scene_00_2','scene_00_1','scene_01_0','scene_01_1','scene_01_2','scene_01_3']
    image_path_list = []
    for scene in scences:
      image_path_list += glob.glob(os.path.join(DATA_PATH,scene, "pngs", "*.png"))
    train, val = train_test_split(image_path_list, test_size=0.1)

    print("train size ", len(train))
    print("val size ", len(val))
    return train, val



class ListDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, batch_size=1, num_workers=4):
        """
        if you have different image size, then batch_size must be 1
        :param root:
        :param shape:
        :param shuffle:
        :param transform:
        :param train:
        :param seen:
        :param batch_size:
        :param num_workers:
        """
        #if train:
            #root = root *4
        if shuffle:
            random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
        
        #img = 255.0 * F.to_tensor(img)
        
        #img[0,:,:]=img[0,:,:]-92.8207477031
        #img[1,:,:]=img[1,:,:]-95.2757037428
        #img[2,:,:]=img[2,:,:]-104.877445883
        if self.transform is not None:
            img = self.transform(img)
        return img,target


        
class TestNet(nn.Module):
    def __init__(self, pretrained=False):
        super(TestNet, self).__init__()
        
        vgg = models.vgg16_bn(pretrained=pretrained)
        
        self.backend_feat  = [256,128,64]


        # Front End Development VGG - 16 
        features = list(vgg.features.children())
        # get each stage of the VGG - 16
        self.features1 = nn.Sequential(*features[0:6])
        self.features2 = nn.Sequential(*features[6:13])
        self.features3 = nn.Sequential(*features[13:23])
        self.features4 = nn.Sequential(*features[23:33])
        self.features5 = nn.Sequential(*features[33:43])

        # Front End Development P1 to P5 
        self.p5 = nn.Sequential(
            Conv2d(512, 1024, 3, same_padding=True, NL='relu'),
            Conv2d(1024, 512, 3, same_padding=True, NL='relu'),
        )

        self.p4 = nn.Sequential(
            Conv2d(1024, 512, 3, same_padding=True, NL='relu'),
            Conv2d(512, 256, 3, same_padding=True, NL='relu'),
        )

        self.p3 = nn.Sequential(
            Conv2d(512 , 256, 3, same_padding=True, NL='relu'),
            Conv2d(256, 128, 3, same_padding=True, NL='relu'),
        )

        self.p2 = nn.Sequential(
            Conv2d(256, 128, 3, same_padding=True, NL='relu'),
            Conv2d(128, 64, 3, same_padding=True, NL='relu'),
        )

        self.p1 = nn.Sequential(
            Conv2d(128, 64, 3, same_padding=True, NL='relu'),
            Conv2d(64, 64, 3, same_padding=True, NL='relu'),
        ) 

        # Multi-Branch moules
        self.multi_branch5 = nn.Sequential(
            MultiBranchModule(512),
            Conv2d(2048, 1, 1, same_padding=True)
        )

        self.multi_branch4 = nn.Sequential(
            MultiBranchModule(256),
            Conv2d(1024, 1, 1, same_padding=True)
        )

        self.multi_branch3 = nn.Sequential(
            MultiBranchModule(128),
            Conv2d(512, 1, 1, same_padding=True)
        )

        self.multi_branch2 = nn.Sequential(
            MultiBranchModule(64),
            Conv2d(256, 1, 1, same_padding=True)
        )

        self.multi_branch1 = nn.Sequential(
            MultiBranchModule(64),
            Conv2d(256, 1, 1, same_padding=True)
        )

        self.backend = make_layers(self.backend_feat,in_channels = 5,dilation = True)

        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)

    
    
    def forward(self, x):
        size = x.size()
        x1 = self.features1(x)
        x2 = self.features2(x1)
        x3 = self.features3(x2)
        x4 = self.features4(x3)
        x5 = self.features5(x4)

        # Front End Development P1 to P5 
        x = self.p5(x5)
        x5_out = x
        x = F.upsample_bilinear(x, size=x4.size()[2:])

        x = torch.cat([x4, x], 1)
        x = self.p4(x)
        x4_out = x
        x = F.upsample_bilinear(x, size=x3.size()[2:])

        x = torch.cat([x3, x], 1)
        x = self.p3(x)
        x3_out = x
        x = F.upsample_bilinear(x, size=x2.size()[2:])

        x = torch.cat([x2, x], 1)
        x = self.p2(x)
        x2_out = x
        x = F.upsample_bilinear(x, size=x1.size()[2:])

        x = torch.cat([x1, x], 1)
        x = self.p1(x)
        x1_out = x


        # multi-branch predictions
        x5_density = self.multi_branch5(x5_out)
        x4_density = self.multi_branch4(x4_out)
        x3_density = self.multi_branch3(x3_out)
        x2_density = self.multi_branch2(x2_out)
        x1_density = self.multi_branch1(x1_out)

        # upsample the multi-branch predictions to be the same with the input size
        x5_density = F.upsample_nearest(x5_density, size=x1.size()[2:])
        x4_density = F.upsample_nearest(x4_density, size=x1.size()[2:])
        x3_density = F.upsample_nearest(x3_density, size=x1.size()[2:])
        x2_density = F.upsample_nearest(x2_density, size=x1.size()[2:])
        x1_density = F.upsample_nearest(x1_density, size=x1.size()[2:])


        density_map = torch.cat([x5_density, x4_density, x3_density, x2_density, x1_density], 1)


        x_out = self.backend(density_map)
        density_map_out = self.output_layer(x_out)
        return density_map_out
        #return density_map
                
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    dilation_rates = [2,3,5]
    #for v in cfg:
    for v in range(len(cfg)):
        if cfg[v] == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, cfg[v], kernel_size=3, padding=dilation_rates[v],dilation = dilation_rates[v])
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(cfg[v]), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = cfg[v]
    return nn.Sequential(*layers)  

In [49]:
model = TestNet()
model = model.cuda()

In [50]:
path_list_file = "/content/drive/MyDrive/GCC/all_list.txt"
def get_image_path(file_path):
    file_path_list = file_path.split(" ")
    scene = file_path_list[3][4:]
    image_number = file_path_list[4]
    image_path = "/content/drive/MyDrive/GCC/"+"Part"+" "+scene[7]+scene
    return image_path

file = open(path_list_file, 'r')

file_list = file.readlines()

image_path_list = []

for line in file_list:
    image_path_list.append(get_image_path(line))

print(len(image_path_list))


def get_image_pathlist(path_list, part):
    image_path_list_part_ = []
    for line_ in path_list:
        if line_.find(part)!=-1:
            image_path_list_part_.append(line_)
    return image_path_list_part_
part_0_list = get_image_pathlist(image_path_list, "Part 0")
print(len(part_0_list))

15212
1793


In [51]:
train_list, val_list = get_train_val_list("/content/drive/MyDrive/GCC/Part 0")
train_loader = torch.utils.data.DataLoader(ListDataset(train_list,
                                                                shuffle=True,
                                                                transform=transforms.Compose([
                                                                    transforms.ToTensor(), transforms.Resize(480),transforms.Normalize(mean=[0.302234709263, 0.291243076324, 0.269087553024],
                                                                                                                std=[0.227743327618, 0.211051672697, 0.184846073389]),
                                                                ]),
                                                                train=True,
                                                                batch_size=1,
                                                                num_workers=1),batch_size=1)

train size  324
val size  36


In [52]:
test_loader = torch.utils.data.DataLoader(ListDataset(val_list,
                                  shuffle=False,
                                  transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(480),transforms.Normalize(mean=[0.302234709263, 0.291243076324, 0.269087553024],
                                                                                                                  std=[0.227743327618, 0.211051672697, 0.184846073389]),
                                                                  ]), train=False),
              batch_size=1)

In [54]:
lr = 1e-5
criterion = nn.MSELoss(size_average=False).cuda()

#optimizer = torch.optim.SGD(model.parameters(), lr,momentum=0.95,weight_decay=5 * 1e-4)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [55]:
for epoch in range(0,2):
    mae_train = 0
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    model.train()
    end = time.time()

    for i, (img, target) in enumerate(train_loader):

        img = img.cuda()
        img = Variable(img)

        #forward
        output = model(img)

        target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        target = Variable(target)
        #print(target.shape)
        #print(output.shape)

        #backword
        loss = criterion(output, target)
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()


        #calculate the MAE values
        pred_map = output.data.cpu().numpy()
        gt_map = target.data.cpu().numpy()
        mae_train += abs(np.sum(pred_map) -np.sum(gt_map))

        if i % 20 == 0:
            print('Epoch : {}, train loss : {}'.format(epoch, loss))
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                .format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses))
            
    mae_train = mae_train / len(train_loader)
    print(' * TESTING MAE {mae:.3f} '.format(mae=mae_train))
PATH = '/content/drive/MyDrive/GCC/GCC.pth'
torch.save(model.state_dict(), PATH)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3770: UserWarning: nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.")


Epoch : 0, train loss : 60808.46875
Epoch: [0][0/324]	Time 6.422 (6.422)	Data 0.000 (0.000)	Loss 60808.4688 (60808.4688)	
Epoch : 0, train loss : 23415.83203125
Epoch: [0][20/324]	Time 7.155 (6.930)	Data 0.000 (0.000)	Loss 23415.8320 (22739.8360)	
Epoch : 0, train loss : 1451.77587890625
Epoch: [0][40/324]	Time 7.385 (6.874)	Data 0.000 (0.000)	Loss 1451.7759 (35716.5426)	
Epoch : 0, train loss : 32671.890625
Epoch: [0][60/324]	Time 7.107 (6.837)	Data 0.000 (0.000)	Loss 32671.8906 (30087.3963)	
Epoch : 0, train loss : 32300.404296875
Epoch: [0][80/324]	Time 7.049 (6.800)	Data 0.000 (0.000)	Loss 32300.4043 (33299.0906)	
Epoch : 0, train loss : 703.3392944335938
Epoch: [0][100/324]	Time 7.170 (6.783)	Data 0.000 (0.000)	Loss 703.3393 (35122.0315)	
Epoch : 0, train loss : 558.9595947265625
Epoch: [0][120/324]	Time 7.453 (6.767)	Data 0.000 (0.000)	Loss 558.9596 (34704.8080)	
Epoch : 0, train loss : 34753.12109375
Epoch: [0][140/324]	Time 6.299 (6.757)	Data 0.000 (0.000)	Loss 34753.1211 (3556

In [36]:
'/content/drive/MyDrive/GCC'
PATH = '/content/drive/MyDrive/GCC/GCC.pth'
torch.save(model.state_dict(), PATH)

In [56]:
model_path = '/content/drive/MyDrive/GCC/GCC.pth'

model = TestNet().cuda()
# load the trained model
model.load_state_dict(torch.load(model_path))
print('successfully load model from', model_path)


with torch.no_grad():
  model.eval()
  mae = 0

  for i, (img, target) in enumerate(test_loader):
      img = img.cuda()
      img = Variable(img)   
      output = model(img)


      pred_map = output.data.cpu().numpy()
      gt_map = target.data.cpu().numpy()
      print("Model Predicted Count", np.sum(pred_map))
      print("Ground Trueth Count", np.sum(gt_map))

      mae += abs(np.sum(pred_map) -np.sum(gt_map))

  mae = mae / len(test_loader)
  print(' * TESTING MAE {mae:.3f} '.format(mae=mae))

successfully load model from /content/drive/MyDrive/GCC/GCC.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3770: UserWarning: nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.")


Model Predicted Count 32626.963
Ground Trueth Count 34146.996
Model Predicted Count 3591.9443
Ground Trueth Count 1313.111
Model Predicted Count 4361.9785
Ground Trueth Count 19720.186
Model Predicted Count 55314.926
Ground Trueth Count 44810.703
Model Predicted Count 5630.5586
Ground Trueth Count 36030.066
Model Predicted Count 11203.624
Ground Trueth Count 27718.703
Model Predicted Count 39146.984
Ground Trueth Count 27133.748
Model Predicted Count 39274.19
Ground Trueth Count 29751.219
Model Predicted Count 5354.5405
Ground Trueth Count 21848.607
Model Predicted Count 35095.14
Ground Trueth Count 34788.113
Model Predicted Count 15265.731
Ground Trueth Count 4138.9697
Model Predicted Count 3673.7407
Ground Trueth Count 30971.176
Model Predicted Count 13528.125
Ground Trueth Count 4229.5723
Model Predicted Count 35889.105
Ground Trueth Count 29751.383
Model Predicted Count 14277.929
Ground Trueth Count 29101.375
Model Predicted Count 2739.3733
Ground Trueth Count 736.6368
Model Predic

In [59]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.2 GB  | Proc size: 2.6 GB
GPU RAM Free: 9718MB | Used: 1723MB | Util  15% | Total 11441MB


In [58]:
import torch, gc
#with torch.no_grad():
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |     776 MB |   10754 MB |   25591 GB |   25590 GB |\n|       from large pool |     744 MB |   10713 MB |   25555 GB |   25554 GB |\n|       from small pool |      32 MB |      52 MB |      35 GB |      35 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |     776 MB |   10754 MB |   25591 GB |   25590 GB |\n|       from large pool |     744 MB |   10713 MB |